In [17]:
@file:DependsOn("ro.jf.funds:user-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:reporting-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
@file:DependsOn("com.charleskorn.kaml:kaml-jvm:0.67.0")
@file:DependsOn("ro.jf.funds:funds-notebook-client:1.0.0")
@file:DependsOn("io.ktor:ktor-client-core:3.1.3")
@file:DependsOn("io.ktor:ktor-client-cio:3.1.3")
@file:DependsOn("io.ktor:ktor-client-content-negotiation:3.1.3")
@file:DependsOn("io.ktor:ktor-serialization-kotlinx-json:3.1.3")
%use dataframe
%use kandy

import ro.jf.funds.client.notebook.*
import ro.jf.funds.reporting.api.model.*

val client = FundsClient()
val user = client.ensureUserExists("Johann-20.0")

val granularInterval = GranularDateInterval(
    interval = DateInterval(
        LocalDate.parse("2019-01-01"),
        LocalDate.parse("2020-08-31")
    ),
    granularity = TimeGranularity.MONTHLY
)

user

UserTO(id=9ec38f7a-4318-4a16-b96e-8ad6dcaa1184, username=Johann-20.0)

In [18]:
import ro.jf.funds.user.sdk.UserSdk
import kotlinx.coroutines.runBlocking

val REPORT_DATA_CONFIGURATION_YAML_FILE = "../../data/provision/expenses-report-data-configuration.yaml"

In [19]:
import ro.jf.funds.fund.sdk.FundSdk
import ro.jf.funds.fund.api.model.FundName

val fundSdk = FundSdk()
val expenseFund = runBlocking { fundSdk.getFundByName(user.id, FundName("Expenses")) }
expenseFund

FundTO(id=1c59641d-1b07-4676-8fc0-d572e248ec3f, name=Expenses)

In [20]:
import ro.jf.funds.reporting.api.model.*
import kotlinx.serialization.json.Json
import com.charleskorn.kaml.Yaml

fun getReportDataConfiguration(): ReportDataConfigurationTO {
    val dataConfigurationRawJson: String = File(REPORT_DATA_CONFIGURATION_YAML_FILE).readText()
    val dataConfiguration = Yaml.default.decodeFromString(ReportDataConfigurationTO.serializer(), dataConfigurationRawJson)
    return dataConfiguration
}
val dataConfiguration = getReportDataConfiguration()
dataConfiguration

ReportDataConfigurationTO(currency=Currency(value=RON), filter=RecordFilterTO(labels=[basic, home, shopping_services, transport, fun, gifts, development]), groups=[ReportGroupTO(name=basic, filter=RecordFilterTO(labels=[basic])), ReportGroupTO(name=home, filter=RecordFilterTO(labels=[home])), ReportGroupTO(name=shopping_services, filter=RecordFilterTO(labels=[shopping_services])), ReportGroupTO(name=transport, filter=RecordFilterTO(labels=[transport])), ReportGroupTO(name=fun, filter=RecordFilterTO(labels=[fun])), ReportGroupTO(name=gifts, filter=RecordFilterTO(labels=[gifts])), ReportGroupTO(name=development, filter=RecordFilterTO(labels=[development])), ReportGroupTO(name=investment, filter=RecordFilterTO(labels=[investment]))], features=ReportDataFeaturesConfigurationTO(net=NetReportFeatureTO(enabled=true, applyFilter=true), valueReport=GenericReportFeatureTO(enabled=true), groupedNet=GenericReportFeatureTO(enabled=true), groupedBudget=GroupedBudgetReportFeatureTO(enabled=true, dist

In [21]:
import ro.jf.funds.reporting.sdk.ReportingSdk
import kotlinx.coroutines.delay
import ro.jf.funds.commons.model.labelsOf
import ro.jf.funds.commons.model.Currency
import ro.jf.funds.reporting.api.model.ReportViewTaskStatus
import java.time.Instant

val reportingSdk = ReportingSdk()
val reportView = runBlocking {
    val reportViewName = "Expense report"
    val existingReportView = reportingSdk.listReportViews(user.id).items.firstOrNull { it.name == reportViewName }
    if (existingReportView != null) {
        return@runBlocking existingReportView
    }
    val request = CreateReportViewTO(reportViewName, expenseFund.id, dataConfiguration)
    var task: ReportViewTaskTO = reportingSdk.createReportView(user.id, request)
    val timeout = Instant.now().plusSeconds(120)
    while (task.status == ReportViewTaskStatus.IN_PROGRESS && Instant.now().isBefore(timeout)) {
        delay(2000)
        task = reportingSdk.getReportViewTask(user.id, task.taskId)
    }
    if (task.status == ReportViewTaskStatus.COMPLETED) {
        task.report ?: error("No report found on completed report task")
    } else if (task.status == ReportViewTaskStatus.FAILED) {
        throw IllegalStateException("Report view creation failed on task $task")
    } else {
        throw IllegalStateException("Report view creation timed out on task $task")
    }
}
reportView

ReportViewTO(id=b6b2ad41-1d38-46ad-a230-26a20cb9a7ee, name=Expense report, fundId=1c59641d-1b07-4676-8fc0-d572e248ec3f, dataConfiguration=ReportDataConfigurationTO(currency=Currency(value=RON), filter=RecordFilterTO(labels=[basic, home, shopping_services, transport, fun, gifts, development]), groups=null, features=ReportDataFeaturesConfigurationTO(net=NetReportFeatureTO(enabled=true, applyFilter=true), valueReport=GenericReportFeatureTO(enabled=true), groupedNet=GenericReportFeatureTO(enabled=false), groupedBudget=GroupedBudgetReportFeatureTO(enabled=false, distributions=[]), forecast=ForecastReportFeatureTO(enabled=false, inputBuckets=0, outputBuckets=0))))

In [22]:
import ro.jf.funds.reporting.api.model.*

val report = runBlocking {
    reportingSdk.getReportViewData(user.id, reportView.id, granularInterval)
}
report

ReportDataTO(viewId=b6b2ad41-1d38-46ad-a230-26a20cb9a7ee, granularInterval=GranularDateInterval(interval=DateInterval(from=2019-01-01, to=2020-08-31), granularity=MONTHLY), data=[ReportDataItemTO(timeBucket=DateInterval(from=2019-01-01, to=2019-01-31), bucketType=REAL, net=-4468.51, value=ValueReportTO(start=0.0, end=2481.52, min=0.0, max=0.0), groups=[ReportDataGroupItemTO(group=basic, net=-2305.04, allocated=2085.009, left=-220.031), ReportDataGroupItemTO(group=home, net=-500.0, allocated=2085.009, left=1585.009), ReportDataGroupItemTO(group=shopping_services, net=-385.57, allocated=347.5015, left=-38.0685), ReportDataGroupItemTO(group=transport, net=-204.24, allocated=347.5015, left=143.2615), ReportDataGroupItemTO(group=fun, net=-1031.16, allocated=1042.5045, left=11.3445), ReportDataGroupItemTO(group=gifts, net=0.0, allocated=834.0036, left=834.0036000000001), ReportDataGroupItemTO(group=development, net=-42.5, allocated=208.5009, left=166.0009), ReportDataGroupItemTO(group=invest

In [23]:
import org.jetbrains.kotlinx.kandy.ir.Plot

fun plotGroupData(plotTitle: String, groupItemFilter: (ReportDataGroupItemTO) -> Boolean): Plot =
    dataFrameOf(
        "month" to report.data.map<ReportDataItemTO, LocalDate> { it.timeBucket.from },
        "spent" to report.data.map { bucket -> bucket.groups!!.filter(groupItemFilter).map { it.net!!.negate() }.reduce { acc, value -> acc + value } },
        "allocated" to report.data.map { bucket -> bucket.groups!!.filter(groupItemFilter).map { it.allocated!! }.reduce { acc, value -> acc + value } },
        "left" to report.data.map { bucket -> bucket.groups!!.filter(groupItemFilter).map { it.left!! }.reduce { acc, value -> acc + value } },
    )
        .plot {
            x("month")
            line {
                y.constant(0)
            }
            line {
                y("spent")
                color = Color.RED
            }
            line {
                y("allocated")
                color = Color.GREEN
            }
            area {
                y("left")
                borderLine {
                    color = Color.YELLOW
                }
            }
            layout {
                title = plotTitle
                size = 1200 to 600
            }
        }

fun plotGroup(title: String, group: String): Plot =
    plotGroupData(title) { it.group == group }

fun plotGroupsTotal(): Plot =
    plotGroupData("Total expenses") { true }



In [24]:
plotGroupsTotal()

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="Qs9SaZ"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Total expenses"
},
"mapping":{
},
"data":{
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12],
"left":[2481.52,5185.08,11301.259999999998,11526.0160226864,10643.310981436,11868.237464847596,8637.9027559568,-6550.8393119068,-872.2869408013005,4051.7634872546987,6037.1198616282,12394.376169342602,18628.6707953371,24174.782837978102,22045.408532850503,24075.588180919905,17645.084581462612,6743.352806606012,-10496.233404802091,-7818.277116034784,9717.445815978463,10599.92354571011,11145.603550581394,11571.310524660823,11502.721720937352,10908.892631404035],
"spent":[4468.51,4301.32,9171.45,6967.76,9043.57,7024.09,11232.58,25034.218092,4537.4484992,6776.5,9787.68,5529.22,4702.18,5442.03,35747.94,17363.44,18069.43,20971.82,27334.72,15561.74,14318.6790416,15133.7815868,15830.221719033336,16333.766862286115,17234.145767476617,18278.476248099672],
"allocated":[6950.03,7004.88,15287.63,7192.5160226864,8165.19,8255.89,7999.83,9846.81,10219.0435174615,11703.94,11782.32,11887.53,10934.14,10999.55,33626.01,19395.52,11641.82,10066.59,10094.98,18240.71,14216.012793121792,14549.093566093483,14786.189696601274,15036.512171318045,15298.927352261217,15662.659631616318]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"spent"
},
"stat":"identity",
"color":"#ee6666",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"allocated"
},
"stat":"identity",
"color":"#3ba272",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"left"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"month"
}]
},
"spec_id":"29"
};
 var containerDiv = document.getElementById("Qs9SaZ");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M50.229221679903276 389.8149558828524 L50.229221679903276 389.8149558828524 L91.09814220685803 389.8149558828524 L128.0120059086221 389.8149558828524 L168.88092643557684 389.8149558828524 L208.43149468746924 389.8149558828524 L249.300415214424 389.8149558828524 L288.85098346631275 389.8149558828524 L329.71990

In [25]:
plotGroup("Basic Expenses", "basic")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="LMasPG"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Basic Expenses"
},
"mapping":{
},
"data":{
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12],
"left":[-220.031,-627.8770000000001,610.492,330.48648635219195,508.43567405832044,437.06652313012995,-1112.5004299735367,-947.5345559564795,223.61394177855433,897.2378709208888,847.9020696586114,1302.016580569407,1659.0140306499868,2022.384571887949,7430.884105266242,10555.052750343684,11046.72635849535,10008.780326540711,9087.614386321446,8929.321155140975,5334.212345631151,5760.0955459522,6165.333685538143,6608.452986861437,7050.656020719106,7499.959519891533],
"spent":[2305.04,2509.31,3347.92,2293.91,2108.18,2382.69,3709.61,2493.78,1587.87,2253.23,2992.86,2517.62,2158.1,2165.51,2324.86,948.25,1951.84,3154.12,3041.34,3989.72,2423.776666666667,2493.435555555556,2513.452685185185,2473.502075617284,2469.825581918724,2495.802713745285],
"allocated":[2085.009,2101.464,4586.289,2013.904486352192,2286.2532,2311.6492,2159.9541,2658.6387,2759.141749714605,2925.985,2945.58,2971.8825,2514.8522,2529.8965,7733.9823,4073.0592,2444.7822,2113.9839,2119.9458,3830.5491,3246.970037476217,3287.622394789685,3317.758844355492,3348.773748051783,3380.181352056098,3452.29211472744]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"spent"
},
"stat":"identity",
"color":"#ee6666",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"allocated"
},
"stat":"identity",
"color":"#3ba272",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"left"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"month"
}]
},
"spec_id":"32"
};
 var containerDiv = document.getElementById("LMasPG");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M50.59197647333349 453.9831353252372 L50.59197647333349 453.9831353252372 L91.75605181908759 453.9831353252372 L128.93650697009434 453.9831353252372 L170.10058231584844 453.9831353252372 L209.93678426335464 453.9831353252372 L251.10085960911238 453.9831353252372 L290.9370615566186 453.983135325

In [26]:
plotGroup("Home Expenses", "home")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="gZJvHo"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Home Expenses"
},
"mapping":{
},
"data":{
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12],
"left":[1585.009,3077.313,5899.701999999999,7801.307127259648,10011.240551914512,12179.430030350188,14321.854251894001,-1409.5462605182606,1437.1243162239189,4965.268358721518,8559.880459977263,12200.146656178587,15738.735096768212,16952.31489870024,-645.7517428039287,-9411.658477898953,-17853.325708447996,-31102.451018423042,-50755.94793948097,-54541.450076449044,-8704.75959807785,-9549.91659093633,-10759.51533674115,-12369.46498646769,-14416.93262335487,-16929.90493336513],
"spent":[500.0,609.16,1763.9,400.0,400.0,467.23,420.0,18881.0,423.24,456.78,400.0,400.0,400.0,2736.62,29698.21,16136.26,12866.69,17070.89,23488.88,10712.07,9565.803333333333,10327.68361111111,11150.25891203704,12046.11382137346,13016.62330648791,14068.00858202857],
"allocated":[2085.009,2101.464,4586.289,2301.605127259648,2612.8608,2641.8848,2559.9456,3150.9792,3270.09392558768,3979.3396,4005.9888,4041.7602,3936.2904,3959.838,12105.3636,7370.2976,4423.8916,3825.3042,3836.0924,6931.4698,5140.47751046564,5296.34280920547,5406.093076639259,5522.768433025864,5646.185785778019,5788.677067926188]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"spent"
},
"stat":"identity",
"color":"#ee6666",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"allocated"
},
"stat":"identity",
"color":"#3ba272",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"left"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"month"
}]
},
"spec_id":"35"
};
 var containerDiv = document.getElementById("gZJvHo");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M50.229221679903276 190.65960724392738 L50.229221679903276 190.65960724392738 L91.09814220685803 190.65960724392738 L128.0120059086221 190.65960724392738 L168.88092643557684 190.65960724392738 L208.43149468746924 190.65960724392738 L249.300415214424 190.65960724392738 L288.85098346631275 190.65960724392738 L

In [27]:
plotGroup("Transport Expenses", "transport")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="EZ0pJY"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Transport Expenses"
},
"mapping":{
},
"data":{
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12],
"left":[143.2615,285.6555,-164.08299999999994,57.643121588048054,74.54479148809162,304.22894999699304,736.2355632743659,148.0575371689097,736.3207125468194,1788.1953407875428,577.8233282668358,-135.05922246950692,568.5021375432548,1726.8999941778561,5530.999418129746,8245.895436517525,8655.859177924034,9864.0878447456,10689.110004676293,13243.841743655563,5124.372993041797,5490.044016416378,5798.531406052115,6233.590412534221,6764.311215451199,7280.628638610194],
"spent":[204.24,207.85,1214.12,281.75,554.64,348.18,288.04,1474.32,331.47,238.42,2502.33,2020.41,608.51,161.2,230.49,0.0,1218.9,202.81,588.5,0.0,675.2533333333333,703.9019444444444,742.6921064814815,596.0556153549383,477.3594166345165,466.4302013540595],
"allocated":[347.5015,350.244,764.3815,503.476121588048,571.5633,577.9123,719.9847,886.2129,919.713916571535,1287.4334,1296.0552,1307.6283,1312.0968,1319.946,4035.1212,2715.3728,1629.8548,1409.3226,1413.2972,2553.6994,1766.628468047628,1837.204680670636,1883.018954059855,1931.93260023151,1983.957958584136,2039.946388466147]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"spent"
},
"stat":"identity",
"color":"#ee6666",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"allocated"
},
"stat":"identity",
"color":"#3ba272",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"left"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"month"
}]
},
"spec_id":"38"
};
 var containerDiv = document.getElementById("EZ0pJY");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M50.59197647333349 491.52193244026677 L50.59197647333349 491.52193244026677 L91.75605181908759 491.52193244026677 L128.93650697009434 491.52193244026677 L170.10058231584844 491.52193244026677 L209.93678426335464 491.52193244026677 L251.10085960911238 491.52193244026677 L290.9370615566186 491.52193244026677 L332.1

In [28]:
plotGroup("Shopping & Services Expenses", "shopping_services")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="OJqL8j"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Shopping & Services Expenses"
},
"mapping":{
},
"data":{
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12],
"left":[-38.0685,312.1755,466.787,-582.7620386388161,-1029.621962050885,-3028.5936257685303,-2852.403571310938,-2064.383981710171,-1330.119028158216,-741.5424646803725,202.74219861636232,1153.7092128868185,1659.9849275367612,2318.941374210012,3076.007886837301,4045.5187296950517,4627.123506442154,5062.3703565243895,5567.23480870836,6049.857984295689,2640.985791076192,2971.91119267906,3281.365664125679,3537.917619584789,3736.601653476286,3909.65304730458],
"spent":[385.57,0.0,609.77,1481.1,936.99,2494.99,463.18,0.0,82.99,342.57,0.0,0.0,149.99,0.0,1259.78,0.0,0.0,69.0,0.0,429.95,194.5233333333333,203.8177777777778,192.2550925925926,208.276350308642,225.6327128343621,231.9362722372256],
"allocated":[347.5015,350.244,764.3815,431.550961361184,489.9114,495.3534,639.9864,787.7448,817.52348139692,936.3152,942.5856,951.0024,656.0484,659.973,2017.5606,969.776,582.091,503.3295,504.749,912.0355,871.0824734497432,875.5457227874786,870.4815996864352,864.4729329936381,857.2621440764412,874.0299560828114]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"spent"
},
"stat":"identity",
"color":"#ee6666",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"allocated"
},
"stat":"identity",
"color":"#3ba272",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"left"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"month"
}]
},
"spec_id":"41"
};
 var containerDiv = document.getElementById("OJqL8j");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M50.59197647333349 339.3119343374917 L50.59197647333349 339.3119343374917 L91.75605181908759 339.3119343374917 L128.93650697009434 339.3119343374917 L170.10058231584844 339.3119343374917 L209.93678426335464 339.3119343374917 L251.10085960911238 339.3119343374917 L290.9370615566186 339.3119343374917 L332.1011369023727 339.3119343374917 L373.2652122481304 339.311

In [29]:
plotGroup("Fun Expenses", "fun")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="NDWFw5"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Fun Expenses"
},
"mapping":{
},
"data":{
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12],
"left":[11.3445,492.0765,1547.3310000000006,373.35808294923254,-801.0673165532102,272.71571771413926,362.6625177705111,-535.0432452430558,-1167.0870909754906,-523.7676106416649,537.8412748094674,1507.6251023644716,1694.2888812195467,2463.2198205946825,3528.9020045797206,4498.373807189715,5079.924156140635,5584.260776187441,6089.137130580159,6651.760710748412,2995.373246899758,3342.244941722695,3664.412654419725,3924.960269387247,4126.404866639145,4329.081198757444],
"spent":[1031.16,570.0,1237.89,2109.0,2235.76,0.0,790.09,1980.318092,1753.8784992,405.5,0.0,100.0,579.01,0.0,1287.38,0.0,0.0,0.0,0.0,350.0,372.9807082666667,257.9058923555556,245.6063833851852,266.0735820006173,279.9130471673354,254.9883010979467],
"allocated":[1042.5045,1050.732,2293.1445,935.027082949232,1061.4747,1073.2657,879.9813,1083.1491,1124.094786920765,1053.3546,1060.4088,1069.8777,765.3898,769.9685,2353.8207,969.776,582.091,503.3295,504.749,912.0355,972.4079905767304,959.7674242147275,951.9684928992882,942.9318006408955,932.3529756943035,946.2665736688288]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"spent"
},
"stat":"identity",
"color":"#ee6666",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"allocated"
},
"stat":"identity",
"color":"#3ba272",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"left"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"month"
}]
},
"spec_id":"44"
};
 var containerDiv = document.getElementById("NDWFw5");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M50.59197647333349 426.620439665598 L50.59197647333349 426.620439665598 L91.75605181908759 426.620439665598 L128.93650697009434 426.620439665598 L170.10058231584844 426.620439665598 L209.93678426335464 426.620439665598 L251.10085960911238 426.620439665598 L290.9370615566186 426.620439665598 L332.1011369023727 426.620439665598 L373.26521224813

In [30]:
plotGroup("Gifts Expenses", "gifts")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="A5fBr3"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Gifts Expenses"
},
"mapping":{
},
"data":{
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12],
"left":[834.0036000000001,1282.5892,2969.2547999999992,3430.356722722368,2478.8923100115617,2137.998225902901,-2543.245372581663,-1664.6512660939375,-891.7730111851733,255.1556970194996,-2459.686929890499,-1761.6946269774503,-1325.84106280479,-445.2786348644952,3111.7995828108506,5046.7585403631765,4321.752179989084,4954.933888698035,5849.4946906908945,7776.537726534533,2036.013170031972,2279.995351800067,2448.731989698448,2857.766899664194,3242.722026884331,3623.435617691757],
"spent":[0.0,392.0,147.85,402.0,1930.0,1330.0,5561.66,204.8,351.0,20.0,3892.49,491.19,766.57,330.2,141.92,198.28,2005.0,475.0,216.0,80.0,747.3041666666667,780.3295138888889,843.690306712963,589.6236656057099,597.8264710728524,583.7645103289234],
"allocated":[834.0036,840.5856,1834.5156,863.101922722368,979.8228,990.7068,879.9813,1083.1491,1124.094786920765,1170.394,1178.232,1188.753,1202.7554,1209.9505,3698.8611,2133.5072,1280.6002,1107.3249,1110.4478,2006.4781,1534.283248910064,1568.465620742505,1601.638255804381,1636.922110454746,1674.269536325975,1713.562381019806]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"spent"
},
"stat":"identity",
"color":"#ee6666",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"allocated"
},
"stat":"identity",
"color":"#3ba272",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"left"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"month"
}]
},
"spec_id":"47"
};
 var containerDiv = document.getElementById("A5fBr3");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M50.59197647333349 380.5934911405503 L50.59197647333349 380.5934911405503 L91.75605181908759 380.5934911405503 L128.93650697009434 380.5934911405503 L170.10058231584844 380.5934911405503 L209.93678426335464 380.5934911405503 L251.10085960911238 380.5934911405503 L290.9370615566186 380.5934911405503 L332.101136902372

In [31]:
plotGroup("Development Expenses", "development")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="gFctcX"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Development Expenses"
},
"mapping":{
},
"data":{
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12],
"left":[166.0009,363.1473,-28.223800000000015,115.626520453728,-599.1130674323914,-434.6083564782243,-274.7002031159408,-77.73753955380533,119.63321896828677,-2588.7837048727133,-2229.382539809841,-1872.3675332097253,-1366.01321557587,-863.6991867281423,12.567278030571662,1095.6473947097056,1767.0249109193496,2371.370632332877,2977.1235137017266,4071.8536400390885,291.2478673754428,305.5490880760392,546.7434874884352,778.0873230966249,998.9585611221542,1196.039542513656],
"spent":[42.5,13.0,850.0,0.0,878.0,1.0,0.0,0.0,7.0,3060.0,0.0,0.0,40.0,48.5,805.3,80.65,27.0,0.0,0.0,0.0,339.0375,366.7072916666667,142.2662326388889,154.121752025463,166.9652313609182,177.5456673076614],
"allocated":[208.5009,210.1464,458.6289,143.850320453728,163.3038,165.1178,159.9966,196.9362,204.38087034923,351.1182,353.4696,356.6259,546.707,549.9775,1681.3005,1163.7312,698.5092,603.9954,605.6988,1094.4426,684.1630641957692,724.1449136829808,755.2304731565625,788.7105459196094,824.7175997462435,847.8851497250971]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"spent"
},
"stat":"identity",
"color":"#ee6666",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"allocated"
},
"stat":"identity",
"color":"#3ba272",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"left"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"month"
}]
},
"spec_id":"50"
};
 var containerDiv = document.getElementById("gFctcX");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M50.59197647333349 313.2303780956656 L50.59197647333349 313.2303780956656 L91.75605181908759 313.2303780956656 L128.93650697009434 313.2303780956656 L170.10058231584844 313.2303780956656 L209.93678426335464 313.2303780956656 L251.10085960911238 313.2303780956656 L290.9370615566186 313.2303780956656 L332.1011369023727 313.2303780956656 L373.2652122481304 313.

In [32]:
plotGroup("Investment Expenses", "investment")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="HclcAI"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Investment Expenses"
},
"mapping":{
},
"data":{
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12],
"left":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
"spent":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
"allocated":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"spent"
},
"stat":"identity",
"color":"#ee6666",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"allocated"
},
"stat":"identity",
"color":"#3ba272",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"left"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"month"
}]
},
"spec_id":"53"
};
 var containerDiv = document.getElementById("HclcAI");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M51.317486060193914 260.5 L51.317486060193914 260.5 L93.07187104355035 260.5 L130.78550909303885 260.5 L172.53989407639529 260.5 L212.9473634151327 260.5 L254.70174839848914 260.5 L295.1092177372266 260.5 L336.86360272058664 260.5 L378.6179877039431 260.5 L419.0254570426805 260.5 L460.77984202603693 260.5 L501.18731136477436 260.5 L542.9416963481308 260.5 L584.6960813314909 260.5 L623.756635025602 260.5 L665.5110200089621 260.5 L705.9184893476959 260.5 L747.6728743310559 260.5 L788.0803436697897 260.5 L829.8347286531498 260.5 L871.5891136365099 260.5 L911.9965829752437 260.5 L953.7509679586037 260.5 L994.1584372973375 260.5 L1035.9128222806976 260.5 L1077.6672072640577 260.5 " fill="none" stroke-width="1.6500000000000001" stroke="rgb(71,71,71)" stroke-opacity="1.0">
 
 
 
 
 
 <path d="M51.317486060193914 260.5 L51.317486060193914 260.5 L93.07187104355035 260.5 L130.78550909303885 260.5 L172.53989407639529 260.5 L212.9473634151327 260.5 L254.70174839848914 260.5 L295.1092177372266 260.5 L336.86360272058664 260.5